##**Loading Data From File**  
np.loadtxt를 활용하여 데이터셋을 불러옴

In [2]:
import numpy as np
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

**Slicing**  
파이썬의 강력한 도구 중 하나로 데이터 가져올 때 유용하게 쓰임  
**num[a:b]는 a번째 인덱스에서 b-1번째 인덱스까지 가져옴(비워두면 해당방향으로 전부)**  
예를 들어 num을 [0,1,2,3,4]라 하면  
* num[2:4]는 [2,3]   
* num[:]는 [0,1,2,3,4] (전부)  
* num[1:]는 [1,2,3,4]  
* num[0:-1]는 [0,1,2,3] **(-를 사용하여 역방향으로도 가능)**  

슬라이싱된 부분안에 다른 (슬라이싱 된) 리스트 넣을 수도 있음  
* num[2:4] = [8,9] 시 num은 [0,1,8,9,4]

In [5]:
import tensorflow as tf

print(x_data, "\nx_data shape:", x_data.shape)
print(y_data, "\ny_data shape:", y_data.shape)

W = tf.Variable(tf.random.normal([3, 1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

optimizer = tf.optimizers.SGD(learning_rate=1e-5)

for step in range(2001):
    with tf.GradientTape() as tape:
        hypothesis = tf.matmul(x_data, W) + b
        cost = tf.reduce_mean(tf.square(hypothesis - y_data))
    grads = tape.gradient(cost, [W, b])
    optimizer.apply_gradients(zip(grads, [W, b]))
    if step % 50 == 0:
        print(step, "Cost:", cost.numpy(), "\nPrediction:\n", hypothesis.numpy())

print("Your score will be ", (tf.matmul([[100., 70., 101.]], W) + b).numpy())
print("Other scores will be ", (tf.matmul([[60., 70., 110.], [90., 100., 80.]], W) + b).numpy())

[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]
 [ 69.  74.  77.]
 [ 47.  56.  60.]
 [ 87.  79.  90.]
 [ 79.  70.  88.]
 [ 69.  70.  73.]
 [ 70.  65.  74.]
 [ 93.  95.  91.]
 [ 79.  80.  73.]
 [ 70.  73.  78.]
 [ 93.  89.  96.]
 [ 78.  75.  68.]
 [ 81.  90.  93.]
 [ 88.  92.  86.]
 [ 78.  83.  77.]
 [ 82.  86.  90.]
 [ 86.  82.  89.]
 [ 78.  83.  85.]
 [ 76.  83.  71.]
 [ 96.  93.  95.]] 
x_data shape: (25, 3)
[[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]
 [149.]
 [115.]
 [175.]
 [164.]
 [141.]
 [141.]
 [184.]
 [152.]
 [148.]
 [192.]
 [147.]
 [183.]
 [177.]
 [159.]
 [177.]
 [175.]
 [175.]
 [149.]
 [192.]] 
y_data shape: (25, 1)
0 Cost: 14724.235 
Prediction:
 [[47.127167]
 [45.21018 ]
 [50.641285]
 [51.972187]
 [34.38948 ]
 [19.363007]
 [37.576355]
 [26.28154 ]
 [35.846104]
 [25.322216]
 [36.063644]
 [29.37701 ]
 [55.300632]
 [49.779564]
 [35.71052 ]
 [43.934307]
 [47.722282]
 [45.623882]
 [54.988186]
 [49.920677]
 [43.45054

**tf.data**(tf.train.batch, Queue Runners였던 것)     
파일이 클경우 다음 스텝으로 불러옴  
1. 파일들의 이름을 리스트 통해 불러오기만 하고 읽지 않음
2. 읽어올 데이터를 어떻게 처리할 것인지 정하는 함수작성
3. 데이터가 통과할 경로 설정(일종의 파이프라인)
   * dataset = dataset.map(parse_csv).shuffle(1000).batch(10).repeat()
        * .map(parse_csv): 데이터가 지나갈 때 parse_csv각각 적용
        * .shuffle(1000): 메모리에 임시 버퍼를 만들고 순서대로 학습하지 않도록 1000개 꺼내 뒤섞는 로직 추가
        * .batch(10): 데이터를 하나씩 주지 말고, 10개씩 묶어서 내보내는 로직 추가
        * .repeat(): 데이터가 바닥나면, 다시 처음부터 데이터를 리필하는 루프 로직 추가
4.  Iterator 반복자를 통해 앞서 만든 파이프라인을 사용할 수 있는 객체 생성
5.  for문이 돌아가며 next를 통해 **여기서부터 실제로 데이터가 이동**
    * 디스크에서 데이터 읽음
    * map을 통해 읽어온 데이터가 parse함수로 들어감
    * 변환된 데이터들이 메모리상 1000개 버퍼에 저장된 후 무작위 추출
    * 10개씩 묶인 배치 만들어질때까지 대기
    * 최종적으로 만들어진 데이터가 변수에 할당   
6.  연산 및 업데이트

In [8]:
import tensorflow as tf

dataset = tf.data.TextLineDataset(['data-01-test-score.csv']).skip(1)

def parse_csv(line):
    record_defaults = [[0.], [0.], [0.], [0.]]
    fields = tf.io.decode_csv(line, record_defaults)
    x_data = tf.stack(fields[:-1])
    y_data = tf.stack(fields[-1:])
    return x_data, y_data

dataset = dataset.map(parse_csv).shuffle(1000).batch(10).repeat()

data_iter = iter(dataset)

W = tf.Variable(tf.random.normal([3, 1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

optimizer = tf.optimizers.SGD(learning_rate=1e-5)

for step in range(2001):
    x_batch, y_batch = next(data_iter)
    with tf.GradientTape() as tape:
        hypothesis = tf.matmul(x_batch, W) + b
        cost = tf.reduce_mean(tf.square(hypothesis - y_batch))
    grads = tape.gradient(cost, [W, b])
    optimizer.apply_gradients(zip(grads, [W, b]))

    if step % 10 == 0:
        print(step, "Cost:", cost.numpy(), "\nPrediction:\n", hypothesis.numpy())

print("--------------")
print("Your score will be ", 
      (tf.matmul([[100., 70., 101.]], W) + b).numpy())

print("Other scores will be ", 
      (tf.matmul([[60., 70., 110.], [90., 100., 80.]], W) + b).numpy())

0 Cost: 20534.137 
Prediction:
 [[15.773405 ]
 [ 6.8360696]
 [24.507957 ]
 [26.48336  ]
 [19.029165 ]
 [18.714928 ]
 [24.207855 ]
 [21.613903 ]
 [17.736845 ]
 [22.767954 ]]
10 Cost: 31.13876 
Prediction:
 [[156.73457]
 [148.90749]
 [187.07146]
 [192.80789]
 [151.70233]
 [161.0716 ]
 [143.46858]
 [152.53171]
 [188.54227]
 [176.90628]]
20 Cost: 74.05253 
Prediction:
 [[193.55363]
 [170.47874]
 [152.30588]
 [161.73976]
 [144.07007]]
30 Cost: 31.976954 
Prediction:
 [[170.34286]
 [153.19624]
 [163.55338]
 [157.45775]
 [193.71255]
 [146.01903]
 [189.44258]
 [144.21793]
 [102.56235]
 [176.9705 ]]
40 Cost: 17.071459 
Prediction:
 [[179.28311 ]
 [147.33406 ]
 [108.905014]
 [144.63322 ]
 [198.21228 ]
 [159.51944 ]
 [178.43533 ]
 [143.19653 ]
 [154.43796 ]
 [145.49922 ]]
50 Cost: 55.693474 
Prediction:
 [[162.91066]
 [150.58359]
 [154.02856]
 [108.63241]
 [188.75969]]
60 Cost: 29.99299 
Prediction:
 [[107.848145]
 [157.89404 ]
 [187.33954 ]
 [170.11084 ]
 [170.46042 ]
 [163.31953 ]
 [157.16559 ]